In [11]:
import pandas as pd                       #Хорошие результаты
from rapidfuzz import process, fuzz

# Загрузка данных из Excel
pud = pd.read_excel('АК1.xlsx')  # Файл ПУД
competitor = pd.read_excel('Конкурент.xlsx')  # Файл Конкурент

# Проверка наличия нужных колонок
if pud.shape[1] < 2 or competitor.shape[1] < 2:
    raise ValueError("В каждом файле должно быть минимум 2 колонки: название и цена.")

# Переименование колонок для удобства
pud.columns = ['Название ПУД', 'Цена ПУД']
competitor.columns = ['Название Конкурент', 'Цена Конкурент']

# Функция нормализации текста
def normalize(text, stop_words=None):
    import re
    # Если список стоп-слов не задан, создаем общий список
    if stop_words is None:
        stop_words = [
    'товар', 'упаковка', 'шт', 'г', 'кг', 'литр', 'л', 'банка', 
    'бутылка', 'коробка', 'пакет', 'упак', 'пачка', 'мл', 'единица',
    'продукт', 'натуральный', 'сухой', 'свежий', 'мороженый', 
    'консервированный', 'замороженный' , 'вак/уп', 'пл/ст', 'пюр-пак', 'ст/бут ', 'ПЭТ', 'флоу-пак', 'пл/ст',
    'вес', 'ст/банка', 'ст/б', 'м/уп ', 'газ/уп', 'п/амид', 'вак/уп', 'охл', '~0,5кг'
]
    text = str(text).lower()  # Приведение к нижнему регистру
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)  # Удаление всех символов, кроме букв, цифр и пробелов
    text = re.sub(r'(\d+)([а-яa-z]+)', r'\1 \2', text)  # Пробел между числом и текстом
    text = re.sub(r'\s+', ' ', text).strip()  # Удаление лишних пробелов
    text = re.sub(r'(\d+)\s?г\b', r'\1 г', text)  # Пробел перед "г"
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'(\d+)\s?кг\b', lambda x: f"{int(x.group(1)) * 1000} г", text)  # Преобразование "кг" в "г"
    text = re.sub(r'(\d+)\s?л\b', lambda x: f"{int(x.group(1)) * 1000} мл", text)  # Преобразование "л" в "мл"
    # Удаление стоп-слов
    for word in stop_words:
        text = re.sub(rf'\b{word}\b', '', text)
    return text.strip()

# Нормализация названий товаров
pud['Название нормализовано'] = pud['Название ПУД'].apply(normalize)
competitor['Название нормализовано'] = competitor['Название Конкурент'].apply(normalize)

# Сопоставление товаров
matches = []
for i, row in pud.iterrows():
    pud_name = row['Название нормализовано']
    pud_price = row['Цена ПУД']
    
    # Поиск наиболее похожего названия
    result = process.extractOne(pud_name, competitor['Название нормализовано'], scorer=fuzz.token_sort_ratio)
    
    if result:
        matched_name, score, _ = result
        if score > 80:                  # Порог схожести
            competitor_row = competitor[competitor['Название нормализовано'] == matched_name].iloc[0]
            matches.append([
                row['Название ПУД'],                     # Наименование ПУД
                competitor_row['Название Конкурент'],  # Наименование Конкурент
                pud_price,                           # Цена ПУД
                competitor_row['Цена Конкурент'],  # Цена Конкурент
                score                            # Схожесть
            ])

# Создание DataFrame с результатами
result_df = pd.DataFrame(matches, columns=['Наименование ПУД', 'Наименование Конкурент', 'Цена ПУД', 'Цена Конкурент', 'Схожесть'])

# Сохранение результатов в Excel
result_df.to_excel('ЛучшийРезультатПУД1.xlsx', index=False)

print("Сопоставления успешно сохранены в файл 'ЛучшийРезультат.xlsx'")

Сопоставления успешно сохранены в файл 'СопоставленияClass.xlsx'


In [3]:
import pandas as pd                            #Хорошие результаты сопоставление только один ко дному
from rapidfuzz import process, fuzz

# Загрузка данных из Excel
pud = pd.read_excel('АК.xlsx')  # Файл ПУД
competitor = pd.read_excel('Конкурент1.xlsx')  # Файл Конкурент

# Проверка наличия нужных колонок
if pud.shape[1] < 2 or competitor.shape[1] < 2:
    raise ValueError("В каждом файле должно быть минимум 2 колонки: название и цена.")

# Переименование колонок для удобства
pud.columns = ['Название ПУД', 'Цена ПУД']
competitor.columns = ['Название Конкурент', 'Цена Конкурент']

# Функция нормализации текста
def normalize(text, stop_words=None):
    import re
    # Если список стоп-слов не задан, создаем общий список
    if stop_words is None:
        stop_words = [
            'товар','ТОВАР', 'упаковка', 'шт', 'ШТ', 'г', 'кг', 'КГ','литр', 'л', 'банка', 
            'бутылка', 'коробка', 'пакет', 'упак', 'пачка', 'мл', 'единица',
            'продукт', 'натуральный', 'сухой', 'свежий', 'мороженый', 
            'консервированный', 'замороженный', 'вак/уп', 'пл/ст', 'пюр-пак', 
            'ст/бут', 'ПЭТ', 'флоу-пак', 'пл/ст', 'вес', 'ст/банка', 'ст/б', 
            'м/уп', 'газ/уп', 'п/амид', 'вак/уп', 'охл', '~0,5кг', '[м]'
        ]
    text = str(text).lower()  # Приведение к нижнему регистру
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)  # Удаление всех символов, кроме букв, цифр и пробелов
    text = re.sub(r'(\d+)([а-яa-z]+)', r'\1 \2', text)  # Пробел между числом и текстом
    text = re.sub(r'\s+', ' ', text).strip()  # Удаление лишних пробелов
    text = re.sub(r'(\d+)\s?г\b', r'\1 г', text)  # Пробел перед "г"
    text = re.sub(r'\[.*?\]', '', text)
   
    # Удаление стоп-слов
    for word in stop_words:
        text = re.sub(rf'\b{word}\b', '', text)
    return text.strip()

# Нормализация названий товаров
pud['Название нормализовано'] = pud['Название ПУД'].apply(normalize)
competitor['Название нормализовано'] = competitor['Название Конкурент'].apply(normalize)

# Сопоставление товаров
used_competitor_indices = set()  # Хранение индексов уже использованных товаров
matches = []

for i, row in pud.iterrows():
    pud_name = row['Название нормализовано']
    pud_price = row['Цена ПУД']
    
    # Исключение уже использованных наименований
    available_competitors = competitor.loc[~competitor.index.isin(used_competitor_indices)]
    result = process.extractOne(
        pud_name, 
        available_competitors['Название нормализовано'], 
        scorer=fuzz.token_sort_ratio
    )
    
    if result:
        matched_name, score, match_index = result
        if score > 70:  # Порог схожести
            competitor_row = competitor.loc[match_index]
            matches.append([
                row['Название ПУД'],                     # Наименование ПУД
                competitor_row['Название Конкурент'],  # Наименование Конкурент
                pud_price,                           # Цена ПУД
                competitor_row['Цена Конкурент'],  # Цена Конкурент
                score                            # Схожесть
            ])
            used_competitor_indices.add(match_index)
        else:
            # Если нет подходящего товара
            matches.append([
                row['Название ПУД'], None, pud_price, None, 0
            ])
    else:
        matches.append([
            row['Название ПУД'], None, pud_price, None, 0
        ])

# Создание DataFrame с результатами
result_df = pd.DataFrame(matches, columns=['Наименование ПУД', 'Наименование Конкурент', 'Цена ПУД', 'Цена Конкурент', 'Схожесть'])

# Сохранение результатов в Excel
result_df.to_excel('Сопоставления.xlsx', index=False)

print("Сопоставления успешно сохранены в файл 'Сопоставления.xlsx'")


Сопоставления успешно сохранены в файл 'Сопоставления.xlsx'


In [ ]:
import pandas as pd                                               #BERT
from sentence_transformers import SentenceTransformer, util

# Загрузка данных из Excel
pud = pd.read_excel('ПУД.xlsx')  # Файл ПУД
competitor = pd.read_excel('Конкурент.xlsx')  # Файл Конкурент

# Проверка наличия нужных колонок
if pud.shape[1] < 2 or competitor.shape[1] < 2:
    raise ValueError("В каждом файле должно быть минимум 2 колонки: название и цена.")

# Переименование колонок для удобства
pud.columns = ['Название ПУД', 'Цена ПУД']
competitor.columns = ['Название Конкурент', 'Цена Конкурент']

# Функция нормализации текста
def normalize(text, stop_words=None):
    import re
    # Если список стоп-слов не задан, создаем общий список
    if stop_words is None:
        stop_words = [
            'товар', 'упаковка', 'шт', 'г', 'кг', 'литр', 'л', 'банка', 
            'бутылка', 'коробка', 'пакет', 'упак', 'пачка', 'мл', 'единица',
            'продукт', 'натуральный', 'сухой', 'свежий', 'мороженый', 
            'консервированный', 'замороженный', 'вак/уп', 'пл/ст', 'пюр-пак', 'ст/бут', 'пэт',
            'флоу-пак', 'пл/ст', 'вес', 'ст/банка', 'ст/б', 'м/уп', 'газ/уп', 'п/амид', 'вак/уп'
        ]
    text = str(text).lower()  # Приведение к нижнему регистру
    text = re.sub(r'[^a-zа-яё0-9\s]', '', text)  # Удаление всех символов, кроме букв, цифр и пробелов
    text = re.sub(r'(\d+)([а-яa-z]+)', r'\1 \2', text)  # Пробел между числом и текстом
    text = re.sub(r'\s+', ' ', text).strip()  # Удаление лишних пробелов
    text = re.sub(r'(\d+)\s?г\b', r'\1 г', text)  # Пробел перед "г"
    text = re.sub(r'(\d+)\s?кг\b', lambda x: f"{int(x.group(1)) * 1000} г", text)  # Преобразование "кг" в "г"
    text = re.sub(r'(\d+)\s?л\b', lambda x: f"{int(x.group(1)) * 1000} мл", text)  # Преобразование "л" в "мл"
    # Удаление стоп-слов
    for word in stop_words:
        text = re.sub(rf'\b{word}\b', '', text)
    return text.strip()

# Нормализация названий товаров
pud['Название нормализовано'] = pud['Название ПУД'].apply(normalize)
competitor['Название нормализовано'] = competitor['Название Конкурент'].apply(normalize)

# Загрузка модели BERT
model = SentenceTransformer('all-mpnet-base-v2')  #  модель для эмбеддингов

# Генерация эмбеддингов для нормализованных названий
pud_embeddings = model.encode(pud['Название нормализовано'].tolist(), convert_to_tensor=True)
competitor_embeddings = model.encode(competitor['Название нормализовано'].tolist(), convert_to_tensor=True)

# Сопоставление товаров
matches = []
for i, pud_row in pud.iterrows():
    pud_name = pud_row['Название ПУД']
    pud_price = pud_row['Цена ПУД']
    pud_embedding = pud_embeddings[i]

    # Поиск наиболее похожего названия
    cosine_scores = util.cos_sim(pud_embedding, competitor_embeddings)[0]
    best_match_idx = cosine_scores.argmax().item()
    best_score = cosine_scores[best_match_idx].item()

    # Сравниваем с порогом
    if best_score > 0.7:  # Порог косинусного сходства
        competitor_row = competitor.iloc[best_match_idx]
        matches.append([
            pud_name,  # Наименование ПУД
            competitor_row['Название Конкурент'],  # Наименование Конкурент
            pud_price,  # Цена ПУД
            competitor_row['Цена Конкурент'],  # Цена Конкурент
            round(best_score * 100, 2)  # Схожесть в процентах
        ])

# Создание DataFrame с результатами
result_df = pd.DataFrame(matches, columns=['Наименование ПУД', 'Наименование Конкурент', 'Цена ПУД', 'Цена Конкурент', 'Схожесть'])

# Сохранение результатов в Excel
result_df.to_excel('Сопоставления+.xlsx', index=False)

print("Сопоставления успешно сохранены в файл 'Сопоставления.xlsx'")


In [ ]:
import pandas as pd                                                    # SentenceTransformer
from sentence_transformers import SentenceTransformer, util

# Загрузка модели Sentence-BERT
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # Подходит для русского языка

# Параметры
file1 = 'ПУД.xlsx'  # Путь к первому файлу
file2 = 'Конкурент.xlsx'  # Путь ко второму файлу
output_file = 'matched_products.xlsx'  # Имя выходного файла

# Загрузка данных
df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# Проверяем, что данные содержат необходимые столбцы
col1, col2 = 'Product Name 1', 'Product Name 2'  # Переименуйте в соответствие с вашими данными
df1.columns = ['Product Name 1']
df2.columns = ['Product Name 2']

# Преобразование названий товаров в эмбеддинги
embeddings1 = model.encode(df1[col1].tolist(), convert_to_tensor=True)
embeddings2 = model.encode(df2[col2].tolist(), convert_to_tensor=True)

# Расчет сходства
similarities = util.pytorch_cos_sim(embeddings1, embeddings2)

# Поиск наиболее похожих пар
matches = []
for i, sim_row in enumerate(similarities):
    best_match_idx = sim_row.argmax().item()
    best_match_score = sim_row[best_match_idx].item()
    matches.append({
        col1: df1.iloc[i][col1],
        col2: df2.iloc[best_match_idx][col2],
        'Similarity': best_match_score
    })

# Сохранение результата
matched_df = pd.DataFrame(matches)
matched_df = matched_df.sort_values(by='Similarity', ascending=False)  # Сортируем по схожести
matched_df.to_excel(output_file, index=False)

print(f"Сопоставленные данные сохранены в файл: {output_file}")


In [ ]:
import pandas as pd                                                   #RuBERT
from sentence_transformers import SentenceTransformer, util

# Загрузка модели RuBERT
model = SentenceTransformer('DeepPavlov/rubert-base-cased-sentence')  # Подходит для русскоязычных данных

# Параметры
file1 = 'ПУД.xlsx'  # Путь к первому файлу
file2 = 'Конкурент.xlsx'  # Путь ко второму файлу
output_file = 'matched_products_rubert.xlsx'  # Имя выходного файла

# Загрузка данных
df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

# Проверяем, что данные содержат необходимые столбцы
col1, col2 = 'Product Name 1', 'Product Name 2'  # Задайте названия колонок с товарами
df1.columns = [col1]
df2.columns = [col2]

# Преобразование названий товаров в эмбеддинги
embeddings1 = model.encode(df1[col1].tolist(), convert_to_tensor=True)
embeddings2 = model.encode(df2[col2].tolist(), convert_to_tensor=True)

# Расчет сходства
similarities = util.pytorch_cos_sim(embeddings1, embeddings2)

# Поиск уникальных соответствий
used_indices = set()
matches = []

for i, sim_row in enumerate(similarities):
    # Найти наиболее похожий товар
    best_match_idx = sim_row.argmax().item()
    best_match_score = sim_row[best_match_idx].item()

    # Учитывать условие: уникальное сопоставление
    if best_match_idx not in used_indices and best_match_score > 0.7:  # Порог схожести = 0.7
        matches.append({
            col1: df1.iloc[i][col1],
            col2: df2.iloc[best_match_idx][col2],
            'Similarity': best_match_score
        })
        used_indices.add(best_match_idx)
    else:
        # Если нет совпадений, добавляем запись без пары
        matches.append({
            col1: df1.iloc[i][col1],
            col2: None,
            'Similarity': None
        })

# Сохранение результата
matched_df = pd.DataFrame(matches)
matched_df.to_excel(output_file, index=False)

print(f"Сопоставленные данные сохранены в файл: {output_file}")


In [ ]:
import pandas as pd                                               #RoBERTа
from transformers import RobertaTokenizer, RobertaModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Загрузим данные из файлов
ak_df = pd.read_excel("АК.xlsx")  # или .csv, в зависимости от формата
competitor_df = pd.read_excel("Конкурент.xlsx")  # или .csv

# Инициализация модели и токенизатора Roberta
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

# Функция для получения эмбеддингов из модели Roberta
def get_embedding(text):
    tokens = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**tokens)
    embedding = outputs.last_hidden_state.mean(dim=1)  # Используем среднее значение по всем токенам
    return embedding

# Получаем эмбеддинги для товаров из обоих списков
ak_embeddings = [get_embedding(text).numpy() for text in ak_df['Наименование']]
competitor_embeddings = [get_embedding(text).numpy() for text in competitor_df['Наименование']]

# Вычисление косинусного сходства между товарами
similarities = []
for ak_emb in ak_embeddings:
    similarity_scores = [cosine_similarity(ak_emb, competitor_emb)[0][0] for competitor_emb in competitor_embeddings]
    similarities.append(similarity_scores)

# Находим наиболее похожий товар для каждого из АК
max_similarities = [max(similarity_scores) for similarity_scores in similarities]
best_matches = [competitor_df['Наименование'][similarity_scores.index(max_similarity)] for similarity_scores, max_similarity in zip(similarities, max_similarities)]

# Добавляем столбец с результатами в исходный DataFrame
ak_df['Best Match'] = best_matches
ak_df['Similarity'] = max_similarities

# Сохраняем результат в Excel
ak_df.to_excel("Matched_Products.xlsx", index=False)
